In [ ]:
from scatter.brew.core import make_callable, current_version, rollback, scatter, vaporize

num_1 = 42
num_2 = 42 * 24

In [ ]:
@scatter
def add(a: int, b: int) -> int:
    return a + b

In [ ]:
print("Current function version:", current_version("add"))

callable_func = make_callable("add")

print("Function call result:", callable_func(num_1, num_2))

In [ ]:
@scatter
def add(a: int, b: int, c: int) -> int:
    return a + b + c

In [ ]:
# Updated function

callable_func = make_callable("add")
print("Updated function call result:", callable_func(num_1, num_2))
# print("Updated function call result:", callable_func(num_1, num_2, num_1))

In [ ]:
# Rollback

rollback("add")
print("Version after rollback:", current_version("add"))

In [ ]:
# Calling rolledback function
callable_func = make_callable("add")
print("Rolled back function call result:", callable_func(num_1, num_2))

In [ ]:
print("Deleting the function...")
vaporize("add")